# BB84 Quantum key distribution

# Normal state

In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import random

def step1_alice_prepares_photons(n_bits):
    """
    Step 1: Alice randomly chooses photons in both rectilinear and diagonal bases
    """
    print("=== STEP 1: Alice prepares random bits and bases ===")
    
    # Alice chooses random bits and bases
    alice_bits = [random.randint(0, 1) for _ in range(n_bits)]
    alice_bases = [random.randint(0, 1) for _ in range(n_bits)]  # 0=rectilinear (+), 1=diagonal (x)
    
    print(f"Alice's random bits:  {alice_bits}")
    print(f"Alice's random bases: {alice_bases} (0=rectilinear, 1=diagonal)")
    print()
    
    return alice_bits, alice_bases

# Encoding function (alternative approach)
def encode_message(bits, bases):
    """
    Alternative encoding: prepare states directly in chosen basis
    Z-basis (0): |0⟩ or |1⟩
    X-basis (1): |+⟩ = H|0⟩ or |-⟩ = HX|0⟩
    """
    circuits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        # Z-basis: apply X if bit=1
        if basis == 0:
            if bit == 1:
                qc.x(0)
        # X-basis: prepare state by applying H & X
        else:
            if bit == 0:
                qc.h(0)  # |+⟩ state
            else:
                qc.x(0)
                qc.h(0)  # |-⟩ state
        circuits.append(qc)
    return circuits

def step2_alice_sends_photons(alice_bits, alice_bases):
    """
    Step 2: Alice records photon states and sends them to Bob
    """
    print("=== STEP 2: Alice encodes and sends photons ===")
    
    # Use the alternative encoding function
    alice_photons = encode_message(alice_bits, alice_bases)
    
    # Show encoding for each photon
    for i in range(len(alice_bits)):
        basis_name = "Z-basis" if alice_bases[i] == 0 else "X-basis"
        if alice_bases[i] == 0:  # Z-basis
            state_name = "|0⟩" if alice_bits[i] == 0 else "|1⟩"
        else:  # X-basis
            state_name = "|+⟩" if alice_bits[i] == 0 else "|-⟩"
        
        print(f"Photon {i}: bit={alice_bits[i]} → {state_name} in {basis_name}")
    
    print(f"\nAlice sends {len(alice_photons)} photons to Bob")
    print()
    
    return alice_photons

def step3_bob_measures_photons(alice_photons):
    """
    Step 3: Bob receives photons and randomly measures them in rectilinear or diagonal basis
    """
    print("=== STEP 3: Bob measures photons with random bases ===")
    
    n_bits = len(alice_photons)
    bob_bases = [random.randint(0, 1) for _ in range(n_bits)]
    bob_measured_bits = []
    
    print(f"Bob's random bases:   {bob_bases} (0=rectilinear, 1=diagonal)")
    
    simulator = AerSimulator()
    
    for i in range(n_bits):
        # Copy Alice's photon
        qc = alice_photons[i].copy()
        
        # Bob applies his measurement basis
        if bob_bases[i] == 1:  # Diagonal measurement
            qc.h(0)  # Rotate to diagonal basis
        
        # Measure the photon
        qc.measure(0, 0)
        
        # Run simulation
        job = simulator.run(transpile(qc, simulator), shots=1)
        result = job.result()
        counts = result.get_counts()
        measured_bit = int(list(counts.keys())[0])
        
        bob_measured_bits.append(measured_bit)
        
        basis_name = "rectilinear" if bob_bases[i] == 0 else "diagonal"
        print(f"Photon {i}: Bob measures {measured_bit} using {basis_name} basis")
    
    print(f"\nBob's measured bits:  {bob_measured_bits}")
    print()
    
    return bob_bases, bob_measured_bits

def step4_bob_announces_bases(bob_bases):
    """
    Step 4: Bob announces his measurement bases (but not the results)
    """
    print("=== STEP 4: Bob announces his measurement bases ===")
    print(f"Bob publicly announces his bases: {bob_bases}")
    print("(Bob does NOT announce his measurement results)")
    print()
    
    return bob_bases

def step5_alice_announces_matching(alice_bases, bob_bases):
    """
    Step 5: Alice announces which measurements used the correct (matching) bases
    """
    print("=== STEP 5: Alice announces matching bases ===")
    
    matching_indices = []
    for i in range(len(alice_bases)):
        if alice_bases[i] == bob_bases[i]:
            matching_indices.append(i)
    
    print(f"Alice's bases:        {alice_bases}")
    print(f"Bob's bases:          {bob_bases}")
    print(f"Matching positions:   {matching_indices}")
    print()
    
    return matching_indices

def step6_create_shared_key(alice_bits, bob_measured_bits, matching_indices):
    """
    Step 6: Alice and Bob discard mismatched measurements and create bit string
    """
    print("=== STEP 6: Create shared key from matching measurements ===")
    
    alice_key = [alice_bits[i] for i in matching_indices]
    bob_key = [bob_measured_bits[i] for i in matching_indices]
    
    print(f"Alice's key: {alice_key}")
    print(f"Bob's key:   {bob_key}")
    
    # Check for errors
    errors = sum(1 for i in range(len(alice_key)) if alice_key[i] != bob_key[i])
    error_rate = errors / len(alice_key) if alice_key else 0
    
    print(f"Errors: {errors}/{len(alice_key)} = {error_rate:.2%}")
    print()
    
    return alice_key, bob_key, error_rate

def run_bb84_protocol(n_bits=8):
    """
    Run the complete BB84 protocol
    """
    print("🔐 BB84 Quantum Key Distribution Protocol")
    print("=" * 50)
    print()
    
    # Step 1: Alice prepares
    alice_bits, alice_bases = step1_alice_prepares_photons(n_bits)
    
    # Step 2: Alice sends photons
    alice_photons = step2_alice_sends_photons(alice_bits, alice_bases)
    
    # Step 3: Bob measures
    bob_bases, bob_measured_bits = step3_bob_measures_photons(alice_photons)
    
    # Step 4: Bob announces bases
    step4_bob_announces_bases(bob_bases)
    
    # Step 5: Alice announces matching
    matching_indices = step5_alice_announces_matching(alice_bases, bob_bases)
    
    # Step 6: Create shared key
    alice_key, bob_key, error_rate = step6_create_shared_key(alice_bits, bob_measured_bits, matching_indices)
    
    # Summary
    print("=== PROTOCOL COMPLETE ===")
    if error_rate == 0:
        print("✅ Success! Alice and Bob have identical keys.")
    else:
        print(f"⚠️  Error detected: {error_rate:.2%} - possible eavesdropping!")
    
    return alice_key, bob_key, error_rate

# Example usage
if __name__ == "__main__":
    # Run BB84 protocol with 8 bits
    alice_key, bob_key, error_rate = run_bb84_protocol(n_bits=8)
    
    print(f"\nFinal shared key length: {len(alice_key)} bits")
    print(f"Key security: {'SECURE' if error_rate < 0.1 else 'COMPROMISED'}")

/var/folders/p8/wytng6t53dl_gcrnlz01_zn00000gn/T/ipykernel_15683/699658833.py:2: DeprecationWarning: Using Qiskit with Python 3.9 is deprecated as of the 2.1.0 release. Support for running Qiskit with Python 3.9 will be removed in the 2.3.0 release, which coincides with when Python 3.9 goes end of life.
  from qiskit import QuantumCircuit, transpile


🔐 BB84 Quantum Key Distribution Protocol

=== STEP 1: Alice prepares random bits and bases ===
Alice's random bits:  [1, 0, 0, 0, 0, 0, 1, 1]
Alice's random bases: [0, 1, 0, 0, 0, 1, 0, 1] (0=rectilinear, 1=diagonal)

=== STEP 2: Alice encodes and sends photons ===
Photon 0: bit=1 → |1⟩ in Z-basis
Photon 1: bit=0 → |+⟩ in X-basis
Photon 2: bit=0 → |0⟩ in Z-basis
Photon 3: bit=0 → |0⟩ in Z-basis
Photon 4: bit=0 → |0⟩ in Z-basis
Photon 5: bit=0 → |+⟩ in X-basis
Photon 6: bit=1 → |1⟩ in Z-basis
Photon 7: bit=1 → |-⟩ in X-basis

Alice sends 8 photons to Bob

=== STEP 3: Bob measures photons with random bases ===
Bob's random bases:   [1, 0, 0, 0, 1, 0, 0, 1] (0=rectilinear, 1=diagonal)
Photon 0: Bob measures 0 using diagonal basis
Photon 1: Bob measures 1 using rectilinear basis
Photon 2: Bob measures 0 using rectilinear basis
Photon 3: Bob measures 0 using rectilinear basis
Photon 4: Bob measures 1 using diagonal basis
Photon 5: Bob measures 1 using rectilinear basis
Photon 6: Bob measure

### 

# BB84 with Proper Coherent States - Jupyter Notebook Implementation

## Mathematical Background

### Coherent States
A coherent state $|\alpha\rangle$ is defined as:
$$|\alpha\rangle = e^{-|\alpha|^2/2} \sum_{n=0}^{\infty} \frac{\alpha^n}{\sqrt{n!}} |n\rangle$$

### Photon Number Distribution
For weak coherent states with average photon number $\mu = |\alpha|^2$:
$$P(n) = \frac{\mu^n e^{-\mu}}{n!}$$

### BB84 Encoding with Coherent States
- **Z-basis**: $\alpha_0 = +\sqrt{\mu}$ (bit=0), $\alpha_1 = -\sqrt{\mu}$ (bit=1)
- **X-basis**: $\alpha_0 = +i\sqrt{\mu}$ (bit=0), $\alpha_1 = -i\sqrt{\mu}$ (bit=1)

### Homodyne Detection
Quadrature measurement: $\langle X_\phi \rangle = \sqrt{2\mu} \cos(\arg(\alpha) - \phi_{LO})$

---

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from collections import Counter

def sample_photon_number(mu):
    """Sample from Poisson(μ) distribution"""
    return np.random.poisson(mu)

def create_wcs_pulse(bit, basis, mu):
    """
    Create a weak coherent state pulse for BB84.
    
    No quantum circuits needed - just track the classical information:
    - bit: the information Alice wants to send
    - basis: which BB84 basis Alice chose
    - n_photons: sampled from Poisson(μ)
    """
    n_photons = sample_photon_number(mu)
    
    return {
        'bit': bit,
        'basis': basis,
        'photon_count': n_photons,
        'mu': mu
    }

## Alice's Preparation Phase

Alice needs to:
1. Choose random bits and bases
2. Encode each bit+basis combination as a coherent state amplitude
3. Create proper coherent state representations (not circuits!)

---

In [3]:
def alice_prepares_wcs(bits, bases, mu_signal: float, mu_decoy: list[float] = None, probabilities: list[float] = None, verbose=True):
    """
    Alice prepares weak coherent state pulses with decoy states.
    
    Args:
        bits: List of bits to encode (0 or 1)
        bases: List of bases to use (0=Z-basis, 1=X-basis)
        mu_signal: Intensity for signal states (used for key generation)
        mu_decoy: List of decoy intensities (for security analysis)
        probabilities: Probability of using each decoy intensity (must sum < 1)
        verbose: Print detailed information
    
    Returns:
        wcs_pulses: List of pulse dictionaries
        photon_counts: List of photon counts for each pulse
        intensity_labels: List indicating which intensity was used for each pulse
    """
    
    # Validate inputs
    if mu_decoy is None:
        mu_decoy = []
    if probabilities is None:
        probabilities = []
    
    if len(mu_decoy) != len(probabilities):
        raise ValueError(f"mu_decoy length ({len(mu_decoy)}) must match probabilities length ({len(probabilities)})")
    
    if sum(probabilities) >= 1:
        raise ValueError(f"Sum of probabilities ({sum(probabilities):.3f}) must be less than 1")
    
    # Calculate signal state probability
    prob_signal = 1 - sum(probabilities)
    
    if verbose:
        print(f"=== Alice prepares WCS pulses with decoy states ===")
        print(f"Signal intensity: μ_s = {mu_signal:.3f} (probability: {prob_signal:.3f})")
        for i, (mu_d, prob_d) in enumerate(zip(mu_decoy, probabilities)):
            print(f"Decoy {i+1} intensity: μ_d{i+1} = {mu_d:.3f} (probability: {prob_d:.3f})")
        print()
    
    wcs_pulses = []
    photon_counts = []
    intensity_labels = []  # Track which intensity was used: 'signal', 'decoy_0', 'decoy_1', etc.
    
    # Statistics for each intensity
    intensity_stats = {
        'signal': {'count': 0, 'mu': mu_signal, 'photons': []}
    }
    for i, mu_d in enumerate(mu_decoy):
        intensity_stats[f'decoy_{i}'] = {'count': 0, 'mu': mu_d, 'photons': []}
    
    for i, (bit, basis) in enumerate(zip(bits, bases)):
        # Randomly choose intensity based on probabilities
        rand = random.random()
        cumulative_prob = 0
        chosen_intensity = mu_signal
        intensity_label = 'signal'
        
        # Check decoy states first
        for j, prob_d in enumerate(probabilities):
            cumulative_prob += prob_d
            if rand < cumulative_prob:
                chosen_intensity = mu_decoy[j]
                intensity_label = f'decoy_{j}'
                break
        # If not decoy, use signal (remaining probability)
        
        # Create pulse with chosen intensity
        pulse = create_wcs_pulse(bit, basis, chosen_intensity)
        
        wcs_pulses.append(pulse)
        photon_counts.append(pulse['photon_count'])
        intensity_labels.append(intensity_label)
        
        # Update statistics
        intensity_stats[intensity_label]['count'] += 1
        intensity_stats[intensity_label]['photons'].append(pulse['photon_count'])
        
        # Show first few pulses
        if verbose and i < 8:
            basis_name = "Z-basis" if basis == 0 else "X-basis"
            state_name = {(0,0): "|0⟩", (1,0): "|1⟩", (0,1): "|+⟩", (1,1): "|-⟩"}[(bit, basis)]
            print(f"Pulse {i}: bit={bit} → {state_name} ({basis_name}), "
                  f"μ={chosen_intensity:.3f} ({intensity_label}), n={pulse['photon_count']} photons")
    
    if verbose:
        print(f"\nIntensity distribution:")
        total_pulses = len(bits)
        for label, stats in intensity_stats.items():
            count = stats['count']
            mu = stats['mu']
            actual_prob = count / total_pulses
            avg_photons = np.mean(stats['photons']) if stats['photons'] else 0
            
            print(f"  {label}: {count} pulses ({actual_prob:.1%}), "
                  f"μ={mu:.3f}, avg_photons={avg_photons:.3f}")
        
        # Overall statistics
        actual_avg = np.mean(photon_counts)
        expected_avg = prob_signal * mu_signal + sum(p * mu for p, mu in zip(probabilities, mu_decoy))
        
        print(f"\nOverall photon statistics:")
        print(f"  Expected average photons: {expected_avg:.3f}")
        print(f"  Actual average photons: {actual_avg:.3f}")
        
        # Show distribution
        counter = Counter(photon_counts)
        print(f"  Photon distribution: {dict(sorted(counter.items()))}")
        print()
    
    return wcs_pulses, photon_counts, intensity_labels

In [4]:
def bb84_measurement_outcome(alice_bit, alice_basis, bob_basis, error_prob=0.02):
    """
    Simulate BB84 measurement outcome without quantum circuits.
    
    BB84 truth table:
    - Same basis: Bob gets Alice's bit (with small error probability)  
    - Different basis: Bob gets random result
    """
    if alice_basis == bob_basis:
        # Matching bases → correct result (with small technical error)
        # 2% technical error rate
        if random.random() < error_prob:
            return 1 - alice_bit  # Bit flip error
        else:
            return alice_bit      # Correct result
    else:
        # Mismatched bases → random result (50/50)
        return random.randint(0, 1)

# Bob measurement phase

In [5]:
def bob_measures_wcs(wcs_pulses, bob_bases, detection_efficiency=1.0, eavesdropping=False, verbose=True):
    """
    Bob measures WCS pulses using single-photon detectors.
    Modified to support decoy state analysis.
    
    Simple logic:
    - n_photons = 0: No detection
    - n_photons ≥ 1: Detection with probability 1-(1-η)^n, then measure
    """
    if verbose:
        print("=== Bob measures WCS pulses ===")
    
    bob_results = []
    detected_pulses = 0
    vacuum_pulses = 0
    received_photon_counts = []
    intensity_labels = []  # Track intensity type for each pulse
    
    for i, (pulse, bob_basis) in enumerate(zip(wcs_pulses, bob_bases)):
        n_photons = pulse['photon_count']
        alice_bit = pulse['bit']
        alice_basis = pulse['basis']
        # If eavesdropping, reduce photon count (Eve steals photons)
        if eavesdropping and n_photons >= 2:
            n_photons_received = n_photons - 1
        else:
            n_photons_received = n_photons
        
        received_photon_counts.append(n_photons_received)
        
        if n_photons_received == 0:
            # Vacuum pulse - no photons to detect
            result = None
            vacuum_pulses += 1
            detection_status = "vacuum - no detection"
        else:
            # Non-vacuum pulse - attempt detection
            detection_prob = 1 - (1 - detection_efficiency)**n_photons_received
            
            if random.random() > detection_prob:
                # Detection failed
                result = None
                detection_status = f"{n_photons_received} photons - detection failed"
            else:
                # Successful detection - perform BB84 measurement
                detected_pulses += 1
                result = bb84_measurement_outcome(alice_bit, alice_basis, bob_basis)
                detection_status = f"{n_photons_received} photons - detected bit {result}"
        
        bob_results.append(result)
        
        if verbose and i < 8:  # Show first 8 measurements
            basis_name = "Z-basis" if bob_basis == 0 else "X-basis"
            print(f"Pulse {i}: {basis_name} → {detection_status}")
    
    if verbose:
        total_pulses = len(wcs_pulses)
        detection_rate = detected_pulses / total_pulses
        vacuum_rate = vacuum_pulses / total_pulses
        
        print(f"\nDetection summary:")
        print(f"  Total pulses: {total_pulses}")
        print(f"  Vacuum pulses: {vacuum_pulses} ({vacuum_rate:.1%})")
        print(f"  Successful detections: {detected_pulses} ({detection_rate:.1%})")
        print()
    
    return bob_results, detected_pulses, vacuum_pulses, received_photon_counts

## Sift key and analyze

In [6]:
def sift_keys_wcs(alice_bits, alice_bases, bob_bases, bob_results, intensity_labels, verbose=True):
    """Perform basis sifting - keep only matching bases with successful detection"""
    if verbose:
        print("=== Basis sifting ===")
    
    matching_indices = []
    for i in range(len(alice_bases)):
        if alice_bases[i] == bob_bases[i] and bob_results[i] is not None:
            matching_indices.append(i)
    
    # Extract ALL sifted data (for security analysis)
    alice_sifted = [alice_bits[i] for i in matching_indices]
    bob_sifted = [bob_results[i] for i in matching_indices]
    sifted_intensity_labels = [intensity_labels[i] for i in matching_indices]
    
    # Extract ONLY signal state bits for the actual key
    signal_indices = [i for i, label in enumerate(sifted_intensity_labels) if label == 'signal']
    alice_key = [alice_sifted[i] for i in signal_indices]
    bob_key = [bob_sifted[i] for i in signal_indices]
    
    if verbose:
        print(f"Total sifted bits: {len(alice_sifted)} (all intensities)")
        print(f"Signal state key: {len(alice_key)} bits")
        print(f"Sifted intensity distribution: {Counter(sifted_intensity_labels)}")
        print(f"Alice's key: {alice_key[:10]}{'...' if len(alice_key) > 10 else ''}")
        print(f"Bob's key: {bob_key[:10]}{'...' if len(bob_key) > 10 else ''}")
        print()
    
    return alice_key, bob_key, matching_indices, sifted_intensity_labels

In [8]:
# Simple test
bits_length = 10000
alice_bits = np.random.randint(0, 2, bits_length)
alice_bases = np.random.randint(0, 2, bits_length)
bob_bases = np.random.randint(0, 2, bits_length)

print("🔐 BB84 Decoy State Protocol Test")
print("=" * 40)
print(f"Initial bits: {bits_length}")
print()

# Create pulses with decoy states
wcs_pulses, photon_counts, intensity_labels = alice_prepares_wcs(
    alice_bits, alice_bases, 
    mu_signal=0.1, 
    mu_decoy=[0.05, 0.01], 
    probabilities=[0.3, 0.1], 
    verbose=False
)

print(f"Alice sent {len(wcs_pulses)} pulses")
print(f"Intensity distribution: {Counter(intensity_labels)}")
print()

# Bob measures
bob_results, detected_pulses, vacuum_pulses, received_photon_counts = bob_measures_wcs(
    wcs_pulses, bob_bases, verbose=False
)

print(f"Bob detected {detected_pulses}/{bits_length} pulses ({detected_pulses/bits_length:.1%})")
print(f"Vacuum pulses: {vacuum_pulses} ({vacuum_pulses/bits_length:.1%})")
print()

# Sift keys
alice_key, bob_key, matching_indices, sifted_intensity_labels = sift_keys_wcs(
    alice_bits, alice_bases, bob_bases, bob_results, intensity_labels, verbose=False
)

print(f"After basis sifting:")
print(f"  Signal state key length: {len(alice_key)} bits")
print(f"  Key rate: {len(alice_key)/bits_length:.1%}")
print(f"  Total sifted bits (all intensities): {len(sifted_intensity_labels)}")
print(f"  Sifted intensity distribution: {Counter(sifted_intensity_labels)}")

# Check for errors
errors = sum(1 for a, b in zip(alice_key, bob_key) if a != b)
qber = errors / len(alice_key) if alice_key else 0
print(f"  QBER: {qber:.1%} ({errors} errors)")

print("\n=== Decoy State Analysis Ready ===")
print("Alice can now announce intensity information for security analysis!")

🔐 BB84 Decoy State Protocol Test
Initial bits: 10000

Alice sent 10000 pulses
Intensity distribution: Counter({'signal': 6062, 'decoy_0': 2962, 'decoy_1': 976})

Bob detected 706/10000 pulses (7.1%)
Vacuum pulses: 9294 (92.9%)

After basis sifting:
  Signal state key length: 288 bits
  Key rate: 2.9%
  Total sifted bits (all intensities): 362
  Sifted intensity distribution: Counter({'signal': 288, 'decoy_0': 69, 'decoy_1': 5})
  QBER: 1.7% (5 errors)

=== Decoy State Analysis Ready ===
Alice can now announce intensity information for security analysis!


# Decoy state